### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [148]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
#school_data.head()
#student_data.head()
school_data_complete.head()

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [149]:
# Total schools
totalSchools=len(school_data["School ID"].unique())
# Total students
totalStudents=len(student_data["Student ID"].drop_duplicates().dropna())
# Total budget
totalBudget=school_data["budget"].sum()

# Average maths score
averageMaths=student_data["maths_score"].mean()
# Average reading score
averageReading=student_data["reading_score"].mean()

# % passing maths (the percentage of students who passed maths)
passMaths_df=student_data["maths_score"] >= 50
passMathsPercent=passMaths_df.sum() / totalStudents * 100

# % passing reading (the percentage of students who passed reading)
passReading_df=student_data["reading_score"] >= 50
passReadingPercent=passReading_df.sum() / totalStudents * 100

# % overall passing (the percentage of students who passed maths AND reading)
passOverall_df=((student_data["maths_score"] >= 50 ) & (student_data["reading_score"] >= 50 ))
passOverallPercent=passOverall_df.sum() / totalStudents * 100

In [150]:
# Create a dataframe to hold the above results
summary=[{"Total schools":totalSchools,"Total students":totalStudents,"Total budget":totalBudget,"Average Maths Score":averageMaths,"Average Reading score":averageReading,"% Passing Maths":passMathsPercent,"% Passing Reading ":passReadingPercent,"% Overall Passing":passOverallPercent}]
summary=pd.DataFrame(summary)

# Give the displayed data cleaner formatting
summary

,Total schools,Total students,Total budget,Average Maths Score,Average Reading score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,24649428,70.338192,69.980138,86.078632,84.426857,72.808272


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [151]:
schoolGroup=school_data_complete.groupby(["school_name"],as_index=False)

#schooltype=schoolGroup["type"].unique()
#each_school_students=schoolGroup["Student ID"].count()
#school_budget=schoolGroup["budget"].unique()
#per_student_budget=school_budget/each_school_students
#avg_maths_score=schoolGroup["maths_score"].mean()
#avg_reading_score=schoolGroup["reading_score"].mean()

school_summary=schoolGroup.agg({'Student ID':'count','type':'unique','budget':'unique','maths_score':'mean','reading_score':'mean'})\
    .rename(columns={"school_name":"School Name","Student ID":"Total Students","type":"School Type","budget":"Total School Budget",\
        "maths_score":"Average Maths Score","reading_score":"Average Reading Score"})

,School Name,Total Students,School Type,Total School Budget,Average Maths Score,Average Reading Score
0,Bailey High School,4976,[Government],[3124928],72.352894,71.008842
1,Cabrera High School,1858,[Independent],[1081356],71.657158,71.359526
2,Figueroa High School,2949,[Government],[1884411],68.698542,69.077993


In [152]:
def per_student_b(x):
    df = x['budget'].unique() / x['Student ID'].count()
    return df

per_student_budget=schoolGroup.apply(per_student_b)

In [153]:
def percent_passing_m(x):
    df = (x['maths_score'] >= 50).sum() / x['Student ID'].count() * 100
    return df

percent_passing_maths=schoolGroup.apply(percent_passing_m)

In [154]:

def percent_passing_r(x):
    df = (x['reading_score'] >= 50).sum() / x['Student ID'].count() * 100
    return df

percent_passing_reading=schoolGroup.apply(percent_passing_r)

In [155]:

def percent_passing_o(x):
    df = ((x['maths_score'] >= 50) & (x['reading_score'] >= 50)).sum() / x['Student ID'].count() * 100
    return df

percent_passing_overall=schoolGroup.apply(percent_passing_o)


In [156]:
#school_summary['Per Student Budget']=per_student_budget
school_summary.insert(4,'Per Student Budget',per_student_budget)
school_summary['% Passing Maths']=percent_passing_maths
school_summary['% Passing Reading']=percent_passing_reading
school_summary['% Passing Overall']=percent_passing_overall
school_summary

,School Name,Total Students,School Type,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
0,Bailey High School,4976,[Government],[3124928],[628.0],72.352894,71.008842,91.639871,87.379421,80.084405
1,Cabrera High School,1858,[Independent],[1081356],[582.0],71.657158,71.359526,90.850377,89.074273,80.785791
2,Figueroa High School,2949,[Government],[1884411],[639.0],68.698542,69.077993,81.654798,82.807731,67.650051
3,Ford High School,2739,[Government],[1763916],[644.0],69.091274,69.572472,82.438846,82.219788,67.469880
4,Griffin High School,1468,[Independent],[917500],[625.0],71.788147,71.245232,91.212534,88.487738,81.335150


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Year

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type